In [1]:
!pip install pyspark

In [46]:
!python -m pip install --user  markupsafe==2.0.1

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StringType, StructType, StructField,IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col,lag,when,lit,sum
from pyspark.sql import functions as bet
from pandas_profiling import ProfileReport

In [37]:
def coding(spark: SparkSession) -> None:
    df = spark.read.csv("C:\\Users\\AG98087\\Downloads\\tickersData\\tickersData.csv",header=True,inferSchema=True)
    df=df.fillna(0)
    df.filter(bet.col("Date").between("2017-11-24 05:30:00","2017-12-23 05:30:00")).select("ISIN","Mnemonic","SecurityType","EndPrice","Date")
    df.select("SecurityType").distinct().show(50)
    #1.How many types of ETF products are available?
    df_etf=df.where(df['SecurityType']=="ETF").select("Mnemonic").distinct().count()
    print("First question Answer:",df_etf)
    #2.What are the Top three heavily traded stocks?
    df_highest_trade=df.withColumn("heavy_trade",row_number().over(Window.orderBy(col("TradedVolume").desc(),col("NumberOfTrades").desc())))
    print("Second Question Answer below:")
    df_highest_trade[df_highest_trade["heavy_trade"]<=3].select("ISIN","Mnemonic","SecurityType","EndPrice","Date","NumberOfTrades","TradedVolume").show()
    #3.Which stock had the highest number of consecutive up days(in terms of end of the day prices)?what were the assumptions/adjustments that led to your solution?
    df_consecutive_ups=df.orderBy(col("Date").asc()).withColumn("max_number",row_number().over(Window.partitionBy(col("Date"),col("SecurityType")).orderBy(col("EndPrice").desc())))
    df_consecutive_ups=df_consecutive_ups[df_consecutive_ups["max_number"]==1]
    #Up Trend
    up=df_consecutive_ups.filter(bet.col("Date").between("2017-11-24 05:30:00","2017-12-23 05:30:00")).select("ISIN","Mnemonic","SecurityType","EndPrice","Date")
    up1=up[up["SecurityType"]=="ETC"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up2=up1.withColumn("final_result",when(up1.EndPrice>up1.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print(up2[up2["final_answer"]==5].count())
    up3=up[up["SecurityType"]=="ETF"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up4=up3.withColumn("final_result",when(up3.EndPrice>up3.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print("Third Question Answer-Which stock had highest number of Consecutive updays:SecurityType:ETF")
    print("Fourth Question Answer-Consecutive updays:SecurityType:ETF,Count:",up4[up4["final_answer"]==5].count())
    up5=up[up["SecurityType"]=="Common stock"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up6=up5.withColumn("final_result",when(up5.EndPrice>up5.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print(up6[up6["final_answer"]==5].count())
    up7=up[up["SecurityType"]=="Other"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up8=up7.withColumn("final_result",when(up7.EndPrice>up7.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print(up8[up8["final_answer"]==5].count())
    up9=up[up["SecurityType"]=="ETN"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up10=up9.withColumn("final_result",when(up9.EndPrice>up9.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print(up10[up10["final_answer"]==5].count())
    #Down Trend
    up11=up[up["SecurityType"]=="ETC"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up12=up11.withColumn("final_result",when(up11.EndPrice<up11.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print("Down trend")
    print(up12[up12["final_answer"]==5].count())
    up13=up[up["SecurityType"]=="ETF"].withColumn("lag_column",lag("EndPrice",offset=1, default=0).over(Window.orderBy(col("Date"))))
    up14=up13.withColumn("final_result",when(up13.EndPrice>up13.lag_column,lit(1)).otherwise(0)).withColumn("final_answer",bet.sum("final_result").over(Window.orderBy(col("Date")).rowsBetween(-4,Window.currentRow)))
    print("Fourth Question Answer- 5 Consecutive updays and Down days:SecurityType:ETF,Count:",up14[up14["final_answer"]==5].count())

In [38]:
if __name__ == "__main__":
    # $example on:init_session$
    spark = SparkSession \
        .builder \
        .appName("Python Spark wellsfargo") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    coding(spark)

+------------+
|SecurityType|
+------------+
|         ETF|
|Common stock|
|         ETC|
|       Other|
|         ETN|
+------------+

First question Answer: 1123
Second Question Answer below:
+------------+--------+------------+--------+-------------------+--------------+------------+
|        ISIN|Mnemonic|SecurityType|EndPrice|               Date|NumberOfTrades|TradedVolume|
+------------+--------+------------+--------+-------------------+--------------+------------+
|DE000A0KRJ36|    OD7L|         ETC|   0.026|2017-12-22 05:30:00|             1|     3664584|
|SE0006027546|    SVAB|Common stock|   0.005|2017-11-20 05:30:00|             3|     3190868|
|SE0006027546|    SVAB|Common stock|    0.01|2017-11-27 05:30:00|             3|     2342000|
+------------+--------+------------+--------+-------------------+--------------+------------+

0
Third Question Answer-Which stock had highest number of Consecutive updays:SecurityType:ETF
Fourth Question Answer-Consecutive updays:SecurityTyp

In [ ]:
#spark.sql.execution.arrow.pyspark.enabled